# Pytorch Lighting training
> on huggingface transformers

In [2]:
# default_exp hf.train

In [11]:
# export
from forgebox.hf.data import IOBES
from forgebox.imports import *
import pytorch_lightning as pl
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [6]:
# !pip install transformers==4.9.1
# !pip install pytorch-lightning==1.3.8
# !pip install tensorflow==2.2.0

## Load model and tokenizer

In [ ]:
# export

# ner model and tokenizer
def ner_model_from(
    name:str, dataset: IOBES
):
    """
    name: from_pretrain(name)
    """
    model = AutoModelForTokenClassification.from_pretrained(
        name,
        num_labels=len(dataset.cates),
    )
    dataset.set_hfconfig(model.config)
    return model

def ner_tokenizer_from(
    name: str
):
    return AutoTokenizer.from_pretrained(
        name, add_prefix_space=True)

## Lightning data module

In [8]:
# export

# ner data module
class NERDataModule(pl.LightningDataModule):
    def __init__(self, train_ds, val_ds, batch_size=32):
        super().__init__()
        self.train_ds = train_ds
        self.val_ds = val_ds
        self.batch_size = batch_size

    def train_dataloader(self):
        return self.train_ds.dataloader(batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return self.val_ds.dataloader(batch_size=self.batch_size*2, shuffle=False)

In [10]:
# export

# ner module
class NERModule(pl.LightningModule):
    """
    PyTorch lightning module for training ner model
    """
    def __init__(
        self, model,
        ):
        """
        model: huggingface transformer model for ner
        """
        super().__init__()
        self.model = model

    def forward(self, batch):
        return self.model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels'])
    
    def training_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs.loss
        self.log("loss", loss)
        self.log("acc", self.calcualte_acc(outputs, batch.labels))
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs.loss
        self.log("val_loss", loss)
        self.log("val_acc", self.calcualte_acc(outputs, batch.labels))
        return loss
    
    def calcualte_acc(self, outputs, labels):
        pred_idx = outputs.logits.argmax(-1)
        mask = torch.ones_like(pred_idx)
        mask[labels==-100]=False
        return (pred_idx[mask]==labels[mask]).float().mean()
    
    def configure_optimizers(self):
        # discriminative learning rate
        param_groups = [
            {'params': self.model.roberta.parameters(), 'lr': 5e-6},
            {'params': self.model.classifier.parameters(), 'lr': 1e-3},
        ]
        optimizer = torch.optim.Adam(param_groups, lr=1e-3)
        return optimizer